# Notebook 5: Comparación de Modelos

Este notebook compara los dos modelos entrenados (Regresión Logística y Random Forest) en términos de métricas de desempeño y concordancia.

Pasos:
1. Carga del dataset preprocesado
2. División en conjuntos de entrenamiento y prueba
3. Entrenamiento y evaluación de Regresión Logística
4. Entrenamiento y evaluación de Random Forest
5. Tabla comparativa de métricas
6. Cálculo del coeficiente Kappa de Cohen
7. Conclusiones breves

## 1. Carga del dataset preprocesado

In [1]:
from google.colab import files
uploaded = files.upload()

Saving dataset_preprocesado.csv to dataset_preprocesado.csv


In [2]:
import pandas as pd
import numpy as np

# Cargamos el dataset procesado
df = pd.read_csv('dataset_preprocesado.csv')
print('Shape del dataset preprocesado:', df.shape)
df.head()

Shape del dataset preprocesado: (924, 32)


,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V22,V23,V24,V25,V26,V27,V28,Amount,Class,Amount.1
0,102572.0,-28.709229,22.057729,0.179011,11.845013,-18.983813,6.474115,-43.557242,-41.044261,-13.320155,...,8.316275,5.466230,0.023854,-1.527145,-0.052266,-5.682338,-0.439134,0.01,1.0,-0.449677
1,10998.0,-0.211134,0.542917,1.526624,-0.445930,-0.163348,-0.274603,0.227532,-0.027924,1.357500,...,0.006634,-0.164857,-0.474453,-0.130723,1.046205,-0.080208,0.001368,39.00,0.0,-0.271375
2,13323.0,-5.454362,8.287421,-12.752811,8.594342,-3.106002,-0.274603,-9.252794,4.245062,-6.329801,...,-0.267172,-0.310804,-1.201685,1.352176,-0.052266,1.574715,0.808725,1.00,1.0,-0.445150
3,55028.0,-0.735658,1.234143,0.731932,1.010075,0.320246,0.859934,0.194436,0.723279,-0.694886,...,0.006634,-0.196655,-0.819615,0.028766,-0.185786,0.297339,0.124236,19.95,0.0,-0.358491
4,84708.0,-10.300820,6.483095,-15.076363,6.554191,-8.880252,-4.471672,-14.900689,0.022167,-4.358441,...,1.041642,-0.682790,0.573544,-1.602389,-0.393521,-0.468893,0.105920,1.00,1.0,-0.445150


## 2. División en conjuntos de entrenamiento y prueba

In [3]:
from sklearn.model_selection import train_test_split

# Definimos X e y
X = df.drop(columns=['Class'])
y = df['Class']

# División estratificada en train y test (80%/20%)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print('Tamaño X_train:', X_train.shape)
print('Tamaño X_test:', X_test.shape)

Tamaño X_train: (739, 31)
Tamaño X_test: (185, 31)


## 3. Entrenamiento y evaluación de Regresión Logística
A continuación, volvemos a realizar GridSearchCV para encontrar los mejores hiperparámetros y evaluar el modelo en el conjunto de prueba.

In [4]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix, precision_score, recall_score, f1_score

# Definimos el modelo base
log_base = LogisticRegression(solver='liblinear', max_iter=1000)

# Parámetros para GridSearch
param_grid_log = {
    'C': [0.01, 0.1, 1, 10, 100],
    'penalty': ['l1', 'l2']
}

grid_search_log = GridSearchCV(
    estimator=log_base,
    param_grid=param_grid_log,
    scoring='roc_auc',
    cv=5,
    n_jobs=-1,
    verbose=1
)

# Ajustamos el GridSearch
grid_search_log.fit(X_train, y_train)

print('Mejores parámetros (Logistic):', grid_search_log.best_params_)
print('Mejor AUC en validación (Logistic):', grid_search_log.best_score_)

# Entrenamos el mejor modelo en todo el set de entrenamiento
best_log = grid_search_log.best_estimator_
best_log.fit(X_train, y_train)

# Predicciones y probabilidades en test
y_pred_log = best_log.predict(X_test)
y_prob_log = best_log.predict_proba(X_test)[:, 1]

# Cálculo de métricas
auc_log = roc_auc_score(y_test, y_prob_log)
prec_log = precision_score(y_test, y_pred_log)
rec_log = recall_score(y_test, y_pred_log)
f1_log = f1_score(y_test, y_pred_log)

print(f"Logistic Test AUC: {auc_log:.4f}")
print(f"Logistic Precision: {prec_log:.4f}")
print(f"Logistic Recall: {rec_log:.4f}")
print(f"Logistic F1-score: {f1_log:.4f}")
print("Matriz de Confusión (Logistic):")
print(confusion_matrix(y_test, y_pred_log))

Fitting 5 folds for each of 10 candidates, totalling 50 fits
Mejores parámetros (Logistic): {'C': 1, 'penalty': 'l1'}
Mejor AUC en validación (Logistic): 0.9790868246995608
Logistic Test AUC: 0.9608
Logistic Precision: 0.9518
Logistic Recall: 0.8587
Logistic F1-score: 0.9029
Matriz de Confusión (Logistic):
[[89  4]
 [13 79]]


## 4. Entrenamiento y evaluación de Random Forest
Procedemos con GridSearchCV para Random Forest y evaluamos métricas en el conjunto de prueba.

In [5]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

# Definimos el modelo base
rf_base = RandomForestClassifier(random_state=42)

# Parámetros para GridSearch
param_grid_rf = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2]
}

grid_search_rf = GridSearchCV(
    estimator=rf_base,
    param_grid=param_grid_rf,
    scoring='roc_auc',
    cv=5,
    n_jobs=-1,
    verbose=1
)

# Ajustamos el GridSearch
grid_search_rf.fit(X_train, y_train)

print('Mejores parámetros (RF):', grid_search_rf.best_params_)
print('Mejor AUC en validación (RF):', grid_search_rf.best_score_)

# Entrenamos el mejor modelo en todo el set de entrenamiento
best_rf = grid_search_rf.best_estimator_
best_rf.fit(X_train, y_train)

# Predicciones y probabilidades en test
y_pred_rf = best_rf.predict(X_test)
y_prob_rf = best_rf.predict_proba(X_test)[:, 1]

# Cálculo de métricas
auc_rf = roc_auc_score(y_test, y_prob_rf)
prec_rf = precision_score(y_test, y_pred_rf)
rec_rf = recall_score(y_test, y_pred_rf)
f1_rf = f1_score(y_test, y_pred_rf)

print(f"RF Test AUC: {auc_rf:.4f}")
print(f"RF Precision: {prec_rf:.4f}")
print(f"RF Recall: {rec_rf:.4f}")
print(f"RF F1-score: {f1_rf:.4f}")
print("Matriz de Confusión (RF):")
print(confusion_matrix(y_test, y_pred_rf))

Fitting 5 folds for each of 36 candidates, totalling 180 fits
Mejores parámetros (RF): {'max_depth': 10, 'min_samples_leaf': 2, 'min_samples_split': 5, 'n_estimators': 100}
Mejor AUC en validación (RF): 0.9774535457338123
RF Test AUC: 0.9735
RF Precision: 0.9873
RF Recall: 0.8478
RF F1-score: 0.9123
Matriz de Confusión (RF):
[[92  1]
 [14 78]]


## 5. Tabla comparativa de métricas
Construimos un DataFrame que agrupe las métricas de ambos modelos.

In [6]:
import pandas as pd

# Datos de métricas
metrics_dict = {
    'Modelo': ['Regresión Logística', 'Random Forest'],
    'AUC': [auc_log, auc_rf],
    'Precisión': [prec_log, prec_rf],
    'Recall': [rec_log, rec_rf],
    'F1-score': [f1_log, f1_rf]
}

df_metrics = pd.DataFrame(metrics_dict)
df_metrics

,Modelo,AUC,Precisión,Recall,F1-score
0,Regresión Logística,0.960846,0.951807,0.858696,0.902857
1,Random Forest,0.973469,0.987342,0.847826,0.912281


## 6. Cálculo del coeficiente Kappa de Cohen
Medimos la concordancia entre las predicciones de ambos modelos sobre el mismo conjunto de prueba.

In [7]:
from sklearn.metrics import cohen_kappa_score

kappa = cohen_kappa_score(y_pred_log, y_pred_rf)
print(f"Coeficiente Kappa de Cohen entre los dos modelos: {kappa:.4f}")

Coeficiente Kappa de Cohen entre los dos modelos: 0.8902
